## Orientações:

* 
* 

## Funções:

In [ ]:
from IPython.display import display, Javascript
import pandas as pd
import os

def popupPositivo():
    display (Javascript("""
    require(
        ["base/js/dialog"], 
        function(dialog) {
            dialog.modal({
                title: 'Automatização das Bases by JV Pinheiro',
                body: 'Alterações feitas e arquivo CSV salvo com sucesso.',
                buttons: {
                    'Clique aqui!': {}
                }
            });
        }
    );
    """))
    
def popupNegativo():
    display (Javascript("""
    require(
        ["base/js/dialog"], 
        function(dialog) {
            dialog.modal({
                title: 'Automatização das Bases by JV Pinheiro',
                body: 'Algo deu erro, entre em contato com o desenvolvedor!',
                buttons: {
                    'Sair': {}
                }
            });
        }
    );
    """))

def tratamentoNina (df):
    # Formatação da data
    df['Data'] = pd.to_datetime(df['Data'])
    df['Data'] = df['Data'].dt.strftime('%d/%m/%Y')

    # Remoção de um único (DDD +) telefone
    df['Telefone'] = df['Telefone'].astype(str).str[-11:]
    
    # Remoção de espaços em branco no Telefone e Telefones sem DDD
    df.dropna(subset = ['Telefone'])
    df['Telefone'] = df['Telefone'].str.strip()
    df = df[(df['Telefone'].str.len() >= 10) & (df['Telefone'] != '55') & (df['Telefone'] != '99999999999')]

    # Remoção de duplicatas com base no telefone
    df = df.drop_duplicates(subset='Telefone')

    return df


def tratamentoRDC (df):
    #Apagar colunas que não são necessárias
    colunas_para_apagar = ['Range Inadimplência', 'Inadimplência em dias', 'QTD_FATURAS_PENDENTES (right)', 'FORMA_PAGAMENTO', 'DT_ADESAO', 'DS_PLANO', 'USUARIO_SOLICIT_CANCELAMENTO', 'SG_REGIONAL', 'CICLO', 'DS_EMAIL_FAKE']
    df = df.drop(columns=colunas_para_apagar)

    # Converter a coluna de datas para um objeto de data e hora
    df['DATA_INADIMPLENCIA'] = pd.to_datetime(df['DATA_INADIMPLENCIA'])

    # Formatar a coluna de datas para o formato dd/mm/aaaa
    df['DATA_INADIMPLENCIA'] = df['DATA_INADIMPLENCIA'].dt.strftime('%d/%m/%Y')

    # Converter a coluna 'Telefone' para uma string
    df['TELEFONE'] = df['TELEFONE'].astype(str)

    # Pegar os últimos 11 caracteres de cada linha na coluna 'Telefone'
    df['TELEFONE'] = df['TELEFONE'].str[-11:]

    # Remover duplicatas com base na coluna 'Telefone'
    df = df.drop_duplicates(subset='TELEFONE')

    # Filtrar as linhas com base no comprimento do número de telefone
    df = df[(df['TELEFONE'] != '') & (df['TELEFONE'].str.len() >= 11)]
    
    return df

def tratamentoEmail (df):
    #Apagar colunas que não são necessárias
    colunas_para_apagar = ['DS_STATUS_ORC', 'GSV prescrito', 'GSV abordado', 'GSV_ORÇADO_PRESCRITO_ABORDADO_APROVADO_MED', 'GSV_ORÇADO_PRESCRITO_MED_EXECUTADO_D+30']
    df = df.drop(columns=colunas_para_apagar)

    # Converter a coluna de datas para um objeto de data e hora
    df['Data'] = pd.to_datetime(df['Data'])

    # Formatar a coluna de datas para o formato dd/mm/aaaa
    df['Data'] = df['Data'].dt.strftime('%d/%m/%Y')

    # Converter a coluna 'Telefone' para uma string
    df['Celular'] = df['Celular'].astype(str)

    # Pegar os últimos 11 caracteres de cada linha na coluna 'Telefone'
    df['Celular'] = df['Celular'].str[-11:]

    # Remover duplicatas com base na coluna 'Telefone'
    df = df.drop_duplicates(subset='Celular')

    # Filtrar as linhas com base no comprimento do número de telefone
    df = df[(df['Celular'] != '') & (df['Celular'].str.len() >= 11)]
    
    return df

## Para Nina:

In [ ]:
# Carregar o arquivo Excel em um DataFrame

caminho_pasta = 'Entrada/Nina'
arquivo_na_pasta = os.listdir(caminho_pasta)
colunas_para_manter = ['Data', 'Cliente', 'Telefone', 'Status']

if arquivo_na_pasta[0] == '':
    print ("Arquivo não encontrado!")
else:
    df = pd.read_excel(arquivo_na_pasta[0], usecols=colunas_para_manter, skiprows=1)

#df.info() 
    

# Salvar o DataFrame em um arquivo CSV
#nome_arquivo_csv = 'teste.csv'
#tratamentoNina(df).to_csv(nome_arquivo_csv, index=False)

# Caminho da pasta onde o arquivo CSV será salvo
pasta_destino = 'Saída'

# Verificar se a pasta existe, senão criar
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

# Caminho completo para o arquivo CSV de destino
nome_arquivo_csv = os.path.join(pasta_destino, 'teste.csv')

# Salvar o DataFrame após a filtragem em um arquivo CSV
tratamentoNina(df).to_csv(nome_arquivo_csv, index=False) 
    
print("Alterações feitas e arquivo CSV salvo com sucesso.")

#Condicional do aviso na Tela
if os.path.exists('Saída/teste.csv'):
    popupPositivo()
else:
    popupNegativo()

### Para RDC:

In [ ]:
# Carregar o arquivo Excel em um DataFrame
nome_arquivo_excel = 'Base RDC Inadimplentes 36 - 42 21082023.xlsx'
if nome_arquivo_excel == '':
    print("Arquivo não encontrado!")
else:
    df = pd.read_excel(nome_arquivo_excel)

# Salvar o DataFrame em um arquivo CSV
nome_arquivo_csv = 'teste.csv'
tratamentoRDC.to_csv(nome_arquivo_csv, index=False)

print("Alterações feitas e arquivo CSV salvo com sucesso.")

## Para E-mail:

In [ ]:
# Carregar o arquivo Excel em um DataFrame
nome_arquivo_excel = '(D_15) Lista Não Agendados 8-25-2023 0100am.xlsx'
if nome_arquivo_excel == '':
    print("Arquivo não encontrado!")
else:
    df = pd.read_excel(nome_arquivo_excel, skiprows=4)

# Salvar o DataFrame em um arquivo CSV
nome_arquivo_csv = 'teste.csv'
df.to_csv(nome_arquivo_csv, index=False)

print("Alterações feitas e arquivo CSV salvo com sucesso.")

## Testes:

In [ ]:
#Testes